<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### 1173. Immediate Food Delivery I

```mysql
-- CASE
/*SELECT ROUND(SUM(immediate)/COUNT(immediate)*100, 2) AS immediate_percentage
FROM (
    SELECT delivery_id,
        CASE WHEN order_date=customer_pref_delivery_date THEN 1 ELSE 0 END AS immediate
    FROM delivery) a*/
-- IF
SELECT ROUND(SUM(imm) / COUNT(delivery_id) * 100, 2) immediate_percentage
FROM (
    SELECT delivery_id, IF(order_date=customer_pref_delivery_date, 1, 0) AS imm
    FROM Delivery) AS x
```

### 1174. Immediate Food Delivery II

```mysql
/*SELECT ROUND(SUM(immediate)/COUNT(immediate)*100, 2) AS immediate_percentage
FROM
(
    SELECT CASE WHEN order_date=customer_pref_delivery_date THEN 1 ELSE 0 END AS immediate
    FROM
    (
        SELECT d.customer_id, d.order_date, d.customer_pref_delivery_date
        FROM delivery d
        JOIN (
            SELECT customer_id, MIN(order_date) AS order_date
            FROM delivery
            GROUP BY customer_id) a
        ON d.customer_id=a.customer_id AND d.order_date=a.order_date) b
    ) c */
-- more concise solution
SELECT ROUND(SUM(lookup.a) * 100 / COUNT(*), 2) AS immediate_percentage
FROM (
    SELECT MIN(order_date) = MIN(customer_pref_delivery_date) AS a
    FROM delivery
    GROUP BY customer_id) as lookup
```

### 1179. Reformat Department Table

```mysql
-- USE group by and case
/* SELECT id,
    SUM(CASE WHEN month='Jan' THEN revenue ELSE NULL END) AS Jan_Revenue,
    SUM(CASE WHEN month='Feb' THEN revenue ELSE NULL END) AS Feb_Revenue,
    SUM(CASE WHEN month='Mar' THEN revenue ELSE NULL END) AS Mar_Revenue,
    SUM(CASE WHEN month='Apr' THEN revenue ELSE NULL END) AS Apr_Revenue,
    SUM(CASE WHEN month='May' THEN revenue ELSE NULL END) AS May_Revenue,
    SUM(CASE WHEN month='Jun' THEN revenue ELSE NULL END) AS Jun_Revenue,
    SUM(CASE WHEN month='Jul' THEN revenue ELSE NULL END) AS Jul_Revenue,
    SUM(CASE WHEN month='Aug' THEN revenue ELSE NULL END) AS Aug_Revenue,
    SUM(CASE WHEN month='Sep' THEN revenue ELSE NULL END) AS Sep_Revenue,
    SUM(CASE WHEN month='Oct' THEN revenue ELSE NULL END) AS Oct_Revenue,
    SUM(CASE WHEN month='Nov' THEN revenue ELSE NULL END) AS Nov_Revenue,
    SUM(CASE WHEN month='Dec' THEN revenue ELSE NULL END) AS Dec_Revenue
FROM department
GROUP BY id */
-- Use group by and if, seem a little faster
SELECT 
    id,
    sum( if(month = 'Jan', revenue, null) ) as "Jan_Revenue",
    sum( if(month = 'Feb', revenue, null) ) as "Feb_Revenue",
    sum( if(month = 'Mar', revenue, null) ) as "Mar_Revenue",
    sum( if(month = 'Apr', revenue, null) ) as "Apr_Revenue",
    sum( if(month = 'May', revenue, null) ) as "May_Revenue",
    sum( if(month = 'Jun', revenue, null) ) as "Jun_Revenue",
    sum( if(month = 'Jul', revenue, null) ) as "Jul_Revenue",
    sum( if(month = 'Aug', revenue, null) ) as "Aug_Revenue",
    sum( if(month = 'Sep', revenue, null) ) as "Sep_Revenue",
    sum( if(month = 'Oct', revenue, null) ) as "Oct_Revenue",
    sum( if(month = 'Nov', revenue, null) ) as "Nov_Revenue",
    sum( if(month = 'Dec', revenue, null) ) as "Dec_Revenue"
FROM Department
GROUP BY id
```

### 1193. Monthly Transactions I

```mysql
SELECT LEFT(trans_date, 7) AS month, country,
    COUNT(*) AS trans_count,
    SUM(CASE WHEN state='approved' THEN 1 ELSE 0 END) AS approved_count,
    SUM(amount) AS trans_total_amount,
    SUM(CASE WHEN state='approved' THEN amount ELSE 0 END) AS approved_total_amount
FROM transactions
GROUP BY month, country
```

### 1194. Tournament Winners

```mysql
-- SQL server
-- use FIRST_VALUE, short code, but slower
/*SELECT DISTINCT group_id,
    FIRST_VALUE(player_id) OVER (PARTITION BY group_id ORDER BY score DESC, player_id) AS player_id
FROM(
    SELECT  group_id, a.first_player AS player_id, SUM(a.first_score) AS score
    FROM (
        SELECT *
        from matches m1
        UNION
        SELECT match_id, second_player AS first_player, first_player AS  second_player,
            second_score AS first_score, first_score AS  second_score
        FROM matches m2) a
    JOIN players p
    ON a.first_player=p.player_id
    GROUP BY group_id, a.first_player) b*/

-- use RANK and rnk=1, faster
SELECT group_id, player_id
FROM (
    SELECT group_id, player_id,
        RANK() OVER (PARTITION BY group_id ORDER BY score DESC, player_id) AS rnk
    FROM(
        SELECT  group_id, a.first_player AS player_id, SUM(a.first_score) AS score
        FROM (
            SELECT *
            from matches m1
            UNION
            SELECT match_id, second_player AS first_player, first_player AS  second_player,
                second_score AS first_score, first_score AS  second_score
            FROM matches m2) a
        JOIN players p
        ON a.first_player=p.player_id
        GROUP BY group_id, a.first_player) b
    ) c
WHERE rnk=1
```

### 1204. Last Person to Fit in the Elevator

```mysql
-- MYSQL
# note have duplicate name, therefore group by id and name
SELECT person_name
FROM (
    SELECT q1.person_id, q1.person_name,
        SUM(q2.weight) AS cum_sum
    FROM queue q1, queue q2
    WHERE q2.turn<=q1.turn
    GROUP BY q1.person_id, q1.person_name
    HAVING cum_sum<=1000
    ) a
ORDER BY cum_sum DESC
LIMIT 1

-- SQL server
SELECT TOP 1 person_name
FROM
    (SELECT person_name,
        SUM(weight) OVER (ORDER BY turn) AS cum_sum
    FROM queue) a
WHERE cum_sum<=1000
ORDER BY cum_sum DESC
```

### 1205. Monthly Transactions II

```mysql
-- MYSQL
-- LEFT JOIN + UNION + RIGHT JOIN === OUTER JOIN
SELECT COALESCE(a.month, b.month) AS month, COALESCE(a.country, b.country) AS country,
    IFNULL(approved_count, 0) AS approved_count, IFNULL(approved_amount, 0) AS approved_amount,
    IFNULL(chargeback_count, 0) AS chargeback_count, IFNULL(chargeback_amount, 0) AS chargeback_amount
FROM (
    SELECT SUBSTR(trans_date, 1, 7) AS month, country,
        COUNT(id) AS approved_count, SUM(amount) AS approved_amount
    FROM transactions
    WHERE state='approved'
    GROUP BY month, country) a
LEFT JOIN (
    SELECT SUBSTR(c.trans_date, 1, 7) AS month, country,
        COUNT(trans_id) AS chargeback_count, SUM(amount) AS chargeback_amount
    FROM transactions t
    JOIN chargebacks c
    ON t.id=c.trans_id
    GROUP BY month, country) b
ON a.month=b.month AND a.country=b.country
UNION
SELECT COALESCE(a.month, b.month) AS month, COALESCE(a.country, b.country) AS country,
    IFNULL(approved_count, 0) AS approved_count, IFNULL(approved_amount, 0) AS approved_amount,
    IFNULL(chargeback_count, 0) AS chargeback_count, IFNULL(chargeback_amount, 0) AS chargeback_amount
FROM (
    SELECT SUBSTR(trans_date, 1, 7) AS month, country,
        COUNT(id) AS approved_count, SUM(amount) AS approved_amount
    FROM transactions
    WHERE state='approved'
    GROUP BY month, country) a
RIGHT JOIN (
    SELECT SUBSTR(c.trans_date, 1, 7) AS month, country,
        COUNT(trans_id) AS chargeback_count, SUM(amount) AS chargeback_amount
    FROM transactions t
    JOIN chargebacks c
    ON t.id=c.trans_id
    GROUP BY month, country) b
ON a.month=b.month AND a.country=b.country

-- SQL server
SELECT COALESCE(a.month, b.month) AS month,
    COALESCE(a.country, b.country) AS country,
    ISNULL(approved_count, 0) AS approved_count,
    ISNULL(approved_amount, 0) AS approved_amount,
    ISNULL(chargeback_count, 0) AS chargeback_count,
    ISNULL(chargeback_amount, 0) AS chargeback_amount
FROM (
    SELECT LEFT(trans_date, 7) AS month, country,
        COUNT(id) AS approved_count,
        SUM(amount) AS approved_amount
    FROM transactions
    WHERE state='approved'
    GROUP BY LEFT(trans_date, 7), country
) a
FULL OUTER JOIN (
    SELECT LEFT(c.trans_date, 7) AS month, country,
        COUNT(trans_id) AS chargeback_count,
        SUM(amount) AS chargeback_amount
    FROM transactions t
    JOIN chargebacks c
    ON t.id=c.trans_id
    GROUP BY LEFT(c.trans_date, 7), country
) b
ON a.month=b.month AND a.country=b.country
```

### 1211. Queries Quality and Percentage

```mysql
SELECT query_name, ROUND(AVG(rating/position), 2) AS quality,
    ROUND(SUM(IF(rating<3, 1, 0))/COUNT(rating)*100, 2) AS poor_query_percentage
FROM queries
GROUP BY query_name
```

### 1212. Team Scores in Football Tournament

```mysql
SELECT t.team_id, t.team_name,
    IFNULL(SUM(points), 0) AS num_points
FROM teams t
LEFT JOIN
(SELECT host_team AS team_id,
    CASE WHEN host_goals>guest_goals THEN 3
        WHEN host_goals=guest_goals THEN 1
        ELSE 0 END as points
FROM matches
UNION ALL
SELECT guest_team AS team_id,
    CASE WHEN host_goals>guest_goals THEN 0
        WHEN host_goals=guest_goals THEN 1
        ELSE 3 END as points
FROM matches) a
ON t.team_id=a.team_id
GROUP BY t.team_id, t.team_name
ORDER BY num_points DESC, t.team_id
```

### 1225. Report Contiguous Dates

```mysql
SELECT *
FROM (
SELECT 'failed' AS period_state, start_date, MIN(end_date) AS end_date
FROM (
    SELECT fail_date AS start_date
    FROM failed
    WHERE fail_date BETWEEN '2019-01-01' AND '2019-12-31'
        AND fail_date NOT IN (
        SELECT ADDDATE(fail_date, INTERVAL 1 DAY)
        FROM failed
        WHERE fail_date BETWEEN '2019-01-01' AND '2019-12-31')
    ) a
CROSS JOIN (
    SELECT fail_date AS end_date
    FROM failed
    WHERE fail_date BETWEEN '2019-01-01' AND '2019-12-31'
        AND fail_date NOT IN (
        SELECT ADDDATE(fail_date, INTERVAL -1 DAY)
        FROM failed
        WHERE fail_date BETWEEN '2019-01-01' AND '2019-12-31')
    ) b
ON start_date<=end_date
GROUP BY start_date
UNION
SELECT 'succeeded' AS period_state, start_date, MIN(end_date) AS end_date
FROM (
    SELECT success_date AS start_date
    FROM succeeded
    WHERE success_date BETWEEN '2019-01-01' AND '2019-12-31'
        AND success_date NOT IN (
        SELECT ADDDATE(success_date, INTERVAL 1 DAY)
        FROM succeeded
        WHERE success_date BETWEEN '2019-01-01' AND '2019-12-31')
    ) a
CROSS JOIN (
    SELECT success_date AS end_date
    FROM succeeded
    WHERE success_date BETWEEN '2019-01-01' AND '2019-12-31'
        AND success_date NOT IN (
        SELECT ADDDATE(success_date, INTERVAL -1 DAY)
        FROM succeeded
        WHERE success_date BETWEEN '2019-01-01' AND '2019-12-31')
    ) b
ON start_date<=end_date
GROUP BY start_date) X
ORDER BY start_date
```

### 1241. Number of Comments per Post

```mysql
SELECT post_id, COUNT(DISTINCT sub_id) AS number_of_comments
FROM (
    SELECT DISTINCT sub_id AS post_id
    FROM submissions
    WHERE parent_id IS NULL) a
LEFT JOIN submissions s
ON post_id=parent_id
GROUP BY post_id
ORDER BY post_id
```

### 1251. Average Selling Price

```mysql
SELECT u.product_id, ROUND(SUM(p.price*u.units)/SUM(u.units), 2) AS average_price
FROM prices p
JOIN (SELECT DISTINCT * FROM unitssold) u
ON p.product_id=u.product_id AND u.purchase_date BETWEEN p.start_date AND p.end_date
GROUP BY u.product_id
```